# Part 1

## 1.Importing essential libraries

In [251]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from pandas.io.json import json_normalize         # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim             # convert an address into latitude and longitude values

import requests                                   # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium                                     # map rendering library

## 2.Scraping the Data from Wikipedia with Pandas

### Copied the Data from the wikipedia page and used the below code the import it in this NoteBook:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [36]:
df_raw = pd.read_clipboard(header=None)

In [37]:
df_raw.shape

(180, 3)

In [38]:
df_raw.head()

,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Assigning the column names:

In [332]:
df = df_raw.copy()

In [333]:
column_names = ['Postal Code','Borough','Neighborhood']

In [334]:
df.columns = column_names

In [335]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Not cleaned data frame shape:

In [336]:
df.shape

(180, 3)

### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned, So we have to remove the rows that the column Borough is 'Not assigned':

In [337]:
df = df[df['Borough'] != 'Not assigned']

In [338]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [339]:
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned',df['Borough'],df['Neighborhood'])

In [340]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### More than one neighborhood can exist in one postal code area. Combining the "Neighborhood" column, separated with a comma, for all the the mutual Postal Codes:

In [341]:
df = df.groupby(['Postal Code','Borough'],as_index=False).agg(','.join)

df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [342]:
df.shape

(103, 3)

# Part 2

### Retrieving the Geographical Coordinates with the provided csv file:

In [343]:
geospatial_data = pd.read_csv('Geospatial_Coordinates.csv')

In [344]:
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging the two DataFrame on the Postal Code column:

In [345]:
df_neighborhoods = pd.merge(df, geospatial_data, on='Postal Code')

In [346]:
df_neighborhoods.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Part 3

##### All the essential packages are imported in the first cell of this Notebook

### The boroughs and Neighborhoods in Toronto

In [347]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
    len(df_neighborhoods['Borough'].unique()),
    df_neighborhoods.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


### Using geopy library to get the latitude and longitude values of Toronto

In [348]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Creating a map of Toronto with neighborhoods superimposed on top

In [349]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neighborhoods['Latitude'], df_neighborhoods['Longitude'], df_neighborhoods['Borough'], df_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='white',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Clustering the map by their borough:

In [350]:
list_boroughs = df_neighborhoods['Borough'].unique()
list_boroughs

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

### Obtaining the Latitude and Longitude of each borough:

In [351]:
borough_dict = {}
for borough in list_boroughs:
    address = (borough + ', Ontario, Canada')
    geolocator = Nominatim(user_agent = 'to_explorer')
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    borough_dict[borough] = [latitude,longitude]

borough_dict

{'Scarborough': [43.7729744, -79.2576479],
 'North York': [43.7543263, -79.44911696639593],
 'East York': [43.699971000000005, -79.33251996261595],
 'East Toronto': [43.72178945, -79.37402706301704],
 'Central Toronto': [43.6534817, -79.3839347],
 'Downtown Toronto': [43.6563221, -79.3809161],
 'York': [43.6896191, -79.479188],
 'West Toronto': [43.6534817, -79.3839347],
 'Mississauga': [43.5896231, -79.6443879],
 'Etobicoke': [43.6435559, -79.5656326]}

In [352]:
df_boroughs = pd.DataFrame(borough_dict).transpose().reset_index()
df_boroughs.columns = ['Borough','Latitude','Longitude']

df_boroughs

,Borough,Latitude,Longitude
0,Scarborough,43.772974,-79.257648
1,North York,43.754326,-79.449117
2,East York,43.699971,-79.332520
3,East Toronto,43.721789,-79.374027
4,Central Toronto,43.653482,-79.383935
5,Downtown Toronto,43.656322,-79.380916
6,York,43.689619,-79.479188
7,West Toronto,43.653482,-79.383935
8,Mississauga,43.589623,-79.644388
9,Etobicoke,43.643556,-79.565633


In [353]:
# create a map of Toronto
map_toronto_boroughs = folium.Map(location = [43.653963, -79.387207], zoom_start = 10)

#add neighborhood markers to the Toronto map
for lat, long, borough in zip(df_boroughs['Latitude'], df_boroughs['Longitude'], df_boroughs['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'white',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto_boroughs)
        
map_toronto_boroughs

### Lets see which boroughs contains the most neighborhoods

In [354]:
df_neighborhoods['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Name: Borough, dtype: int64

### So as we can see 'North York' and 'Downtown Toronto' have the most neighborhoods. I will pick 'Downtown Toronto' for further analysis as it contains many neighborhoods and also it has the word 'Toronto' in it ;)  

In [355]:
# The geographical coordinates for Downtown Toronto
DT_lat = df_boroughs.loc[5,'Latitude']
DT_long = df_boroughs.loc[5,'Longitude']

# The dataframe that contains all the Downtown Toronto neighborhoods
df_DT = df_neighborhoods[df_neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop = True)

# create map of Downtown Toronto neighborhoods using latitude and longitude values
map_DT = folium.Map(location=[DT_lat, DT_long], zoom_start=12.4)

# add markers to map
for lat, lng, label in zip(df_DT['Latitude'], df_DT['Longitude'], df_DT['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='white',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DT)  
    
map_DT

### Define Foursquare Credentials and Version

In [356]:
CLIENT_ID = 'ULPXSFN5LXEZS5WN2ZMBESPQFSQ4WZTGHU1FCL4IQUXA45RQ' # your Foursquare ID
CLIENT_SECRET = '13EIWGTUEQF0QPB2HHHRJNJMM1T0JWXGFRIFDOQWQ3A4DALF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ULPXSFN5LXEZS5WN2ZMBESPQFSQ4WZTGHU1FCL4IQUXA45RQ
CLIENT_SECRET:13EIWGTUEQF0QPB2HHHRJNJMM1T0JWXGFRIFDOQWQ3A4DALF


### Let's explore the first neighborhood in our Downtown Toronto dataframe

In [357]:
df_DT.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [358]:
df_DT.loc[0,'Neighborhood']

'Rosedale'

### Get the neighborhood's latitude and longitude values

In [359]:
neighborhood_latitude = df_DT.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_DT.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_DT.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name,
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


### Now I will get the top 100 venues in Rosedale within a 500 meter radius

#### First, We have to create a GET request

In [360]:
LIMIT = 100

radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=ULPXSFN5LXEZS5WN2ZMBESPQFSQ4WZTGHU1FCL4IQUXA45RQ&client_secret=13EIWGTUEQF0QPB2HHHRJNJMM1T0JWXGFRIFDOQWQ3A4DALF&v=20180605&ll=43.6795626,-79.37752940000001&radius=500&limit=100'

#### Send the GET request and examine the resutls

In [361]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6006925dcb7cea4da4c3a118'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371},
   'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4aff2d47f964a520743522e3',
       'name': 'Rosedale Park',
       'location': {'address': '38 Scholfield Ave.',
        'crossStreet': 'at Edgar Ave.',
        'lat': 43.68232820227814,
        'lng': -79.37893434347683,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.68232820227814,
          'lng': -79.37893434347683}],
        'distance': 32

### Now we have to retrieve, clean and structure the venues data from the JSON format to a pandas DataFrame

In [362]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [363]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-363-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


In [364]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Let's create a function to repeat the same process and retrieve the venues for all the neighborhoods in Downtown Toronto

In [365]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Create a DataFrame called DT_venues that contains all the venues for all the neighborhoods in Downtown Toronto with running the above function

In [366]:
DT_venues = getNearbyVenues(names=df_DT['Neighborhood'],
                                   latitudes=df_DT['Latitude'],
                                   longitudes=df_DT['Longitude'])

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


### Let's check the size of the resulting dataframe

In [367]:
print(DT_venues.shape)
DT_venues.head()

(1222, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


### Let's check how many venues were returned for each neighborhood

In [368]:
DT_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,62,62,62,62,62,62
Christie,15,15,15,15,15,15
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [369]:
print('There are {} uniques categories.'.format(len(DT_venues['Venue Category'].unique())))

There are 208 uniques categories.


# Analyze each neighborhood's venues

In [370]:
# one hot encoding
DT_onehot = pd.get_dummies(DT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DT_onehot['Neighborhood'] = DT_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DT_onehot.columns[-1]] + list(DT_onehot.columns[:-1])
DT_onehot = DT_onehot[fixed_columns]

DT_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [371]:
DT_onehot.shape

(1222, 208)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [372]:
DT_grouped = DT_onehot.groupby('Neighborhood').mean().reset_index()
DT_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.035088,0.000000,0.000000,0.00,0.017544,0.017544,0.000000,0.035088,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.017544,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.000000,0.017544,0.052632,0.087719,0.00000,0.000000,0.000000,0.00000,0.00,0.000000,0.017544,0.000000,0.017544,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.017544,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.017544,0.017544,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.017544,0.017544,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.017544,0.00

In [373]:
DT_grouped.shape

(19, 208)

### Let's print each neighborhood along with the top 5 most common venues

In [374]:
num_top_venues = 5

for hood in DT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DT_grouped[DT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2            Beer Bar  0.04
3  Seafood Restaurant  0.04
4      Farmers Market  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4          Boutique  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1                Café  0.06
2      Sandwich Place  0.06
3  Italian Restaurant  0.05
4     Bubble Tea Shop  0.03


----Christie----
           venue  freq
0  Grocery Store  0.27
1           Café  0.20
2           Park  0.13
3    Candy Store  0.07
4    Coffee Shop  0.07


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.09
1     Sushi Restaurant  0.07
2  Japanese Restaurant  0.07
3              Gay Bar  0.04
4        

### Let's put that into a pandas dataframe

##### First, let's write a function to sort the venues in descending order.

In [375]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [376]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DT_grouped['Neighborhood']

for ind in np.arange(DT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Bakery,Beer Bar,Restaurant,Eastern European Restaurant,Fish Market
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Rental Car Location,Harbor / Marina,Coffee Shop,Sculpture Garden,Airport Gate
2,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Wine Bar,Miscellaneous Shop,Japanese Restaurant
3,Christie,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Nightclub,Comic Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Café,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio
5,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Deli / Bodega
6,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Gym,Japanese Restaurant,Restaurant,American Restaurant,Seafood Restaurant,Asian Restaurant,Salad Place
7,"Garden District, Ryerson",Clothing Store,Coffee Shop,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Hotel,Ramen Restaurant,Lingerie Store
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Restaurant,Italian Restaurant,Scenic Lookout,Brewery,Pizza Place
9,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Mexican Restaurant,Gaming Cafe,Arts & Crafts Store,Caribbean Restaurant,Bar,Bakery


### Cluster the Neighborhoods of Downtown Toronto¶

#### Run k-means to cluster the neighborhoods in Downtown Toronto into 5 clusters

In [377]:
# set number of clusters
kclusters = 5

DT_grouped_clustering = DT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 2, 1, 1, 1, 1, 1, 4])

### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [378]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

DT_merge = df_DT

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DT_merge = DT_merge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

DT_merge.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,3,Park,Trail,Playground,Wine Bar,Creperie,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,1,Coffee Shop,Café,Bakery,Pub,Italian Restaurant,Restaurant,Pizza Place,Pharmacy,Snack Place,Park
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Café,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Theater,Café,Yoga Studio,Dessert Shop,Restaurant
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Hotel,Ramen Restaurant,Lingerie Store


### Finally, let's visualize the resulting clusters

In [379]:
# create map
map_clusters = folium.Map(location=[43.656322, -79.380916], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DT_merge['Latitude'], DT_merge['Longitude'], DT_merge['Neighborhood'], DT_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

##### Cluster 1

In [380]:
DT_merge.loc[DT_merge['Cluster Labels'] == 0, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"CN Tower, King and Spadina, Railway Lands, Har...",0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Rental Car Location,Harbor / Marina,Coffee Shop,Sculpture Garden,Airport Gate


##### Cluster 2

In [381]:
DT_merge.loc[DT_merge['Cluster Labels'] == 1, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"St. James Town, Cabbagetown",1,Coffee Shop,Café,Bakery,Pub,Italian Restaurant,Restaurant,Pizza Place,Pharmacy,Snack Place,Park
2,Church and Wellesley,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Café,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio
3,"Regent Park, Harbourfront",1,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Theater,Café,Yoga Studio,Dessert Shop,Restaurant
4,"Garden District, Ryerson",1,Clothing Store,Coffee Shop,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Hotel,Ramen Restaurant,Lingerie Store
5,St. James Town,1,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Beer Bar,Cosmetics Shop,Lingerie Store,Clothing Store,Department Store
6,Berczy Park,1,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Bakery,Beer Bar,Restaurant,Eastern European Restaurant,Fish Market
7,Central Bay Street,1,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Wine Bar,Miscellaneous Shop,Japanese Restaurant
8,"Richmond, Adelaide, King",1,Coffee Shop,Café,Restaurant,Deli / Bodega,Clothing Store,Thai Restaurant,Gym,Burrito Place,Salad Place,Bakery
9,"Harbourfront East, Union Station, Toronto Islands",1,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Restaurant,Italian Restaurant,Scenic Lookout,Brewery,Pizza Place
10,"Toronto Dominion Centre, Design Exchange",1,Coffee Shop,Hotel,Café,Seafood Restaurant,American Restaurant,Salad Place,Italian Restaurant,Restaurant,Japanese Restaurant,Gastropub


##### Cluster 3

In [382]:
DT_merge.loc[DT_merge['Cluster Labels'] == 2, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Christie,2,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Nightclub,Comic Shop


##### Cluster 4

In [383]:
DT_merge.loc[DT_merge['Cluster Labels'] == 3, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rosedale,3,Park,Trail,Playground,Wine Bar,Creperie,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


##### Cluster 5

In [384]:
DT_merge.loc[DT_merge['Cluster Labels'] == 4, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"University of Toronto, Harbord",4,Café,Bookstore,Bar,Japanese Restaurant,Bakery,Yoga Studio,Beer Bar,Beer Store,Sandwich Place,Italian Restaurant
13,"Kensington Market, Chinatown, Grange Park",4,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Mexican Restaurant,Gaming Cafe,Arts & Crafts Store,Caribbean Restaurant,Bar,Bakery
